# Chapter 17: Metaclasses

Understanding metaclasses, type creation, and advanced class customization



### What are Metaclasses? (Slide 63)


<p><strong>Metaclass</strong> - A class of a class that defines how a class behaves</p>
<p><strong>Key Concept:</strong> Classes are objects too! They're instances of metaclasses.</p>
<ul>
<li>Normal objects are instances of classes</li>
<li>Classes are instances of metaclasses</li>
<li>Default metaclass is <code>type</code></li>
</ul>
<p><strong>The Chain:</strong></p>
<ul>
<li><code>object_instance → MyClass → type</code></li>
<li><code>type(obj)</code> returns the class</li>
<li><code>type(MyClass)</code> returns the metaclass</li>
</ul>
<p><strong>When to Use:</strong></p>
<ul>
<li>Automatic class registration</li>
<li>API frameworks (Django ORM, SQLAlchemy)</li>
<li>Enforcing coding standards</li>
<li>Advanced class customization</li>
</ul>


### type() - Creating Classes Dynamically (Slide 64)


In [1]:
# Normal class definition
class Dog:
    def bark(self):
        return "Woof!"

# Same using type()
Dog2 = type(
    'Dog2',           # Class name
    (),               # Base classes (tuple)
    {'bark': lambda self: "Woof!"}  # Attributes dict
)

# Both are identical
dog1 = Dog()
dog2 = Dog2()
print(dog1.bark())  # Woof!
print(dog2.bark())  # Woof!
print(type(dog1))   # <class '__main__.Dog'>
print(type(Dog))    # <class 'type'>

# With inheritance
class Animal:
    def breathe(self):
        return "Breathing"

Cat = type(
    'Cat',
    (Animal,),  # Inherit from Animal
    {'meow': lambda self: "Meow!"}
)

cat = Cat()
print(cat.breathe())  # Breathing
print(cat.meow())     # Meow!


Woof!
Woof!
<class '__main__.Dog'>
<class 'type'>
Breathing
Meow!


> **Note:** type() is the metaclass of all classes


### Custom Metaclass - Basic (Slide 65)


In [2]:
# Define custom metaclass
class Meta(type):
    def __new__(mcs, name, bases, attrs):
        print(f"Creating class {name}")
        print(f"Bases: {bases}")
        print(f"Attributes: {list(attrs.keys())}")

        # Modify class creation
        attrs['created_by'] = 'Meta'

        # Create the class
        return super().__new__(mcs, name, bases, attrs)

# Use metaclass
class MyClass(metaclass=Meta):
    x = 10
    def method(self):
        pass

# Output during class creation:
# Creating class MyClass
# Bases: ()
# Attributes: ['__module__', '__qualname__', 'x', 'method']

obj = MyClass()
print(obj.created_by)  # Meta
print(type(MyClass))   # <class '__main__.Meta'>


Creating class MyClass
Bases: ()
Attributes: ['__module__', '__qualname__', '__firstlineno__', 'x', 'method', '__static_attributes__', '__classdictcell__']
Meta
<class '__main__.Meta'>


> **Note:** __new__ is called when class is created


### Metaclass __init__ vs __new__ (Slide 66)


In [3]:
class Meta(type):
    def __new__(mcs, name, bases, attrs):
        """Called to CREATE the class object"""
        print(f"__new__: Creating {name}")
        # Must return a class object
        cls = super().__new__(mcs, name, bases, attrs)
        return cls

    def __init__(cls, name, bases, attrs):
        """Called to INITIALIZE the class object"""
        print(f"__init__: Initializing {name}")
        super().__init__(name, bases, attrs)
        # Can modify class after creation
        cls.initialized = True

class MyClass(metaclass=Meta):
    pass

# Output:
# __new__: Creating MyClass
# __init__: Initializing MyClass

print(MyClass.initialized)  # True

# __new__ creates, __init__ initializes
# Similar to regular classes, but at class level!


__new__: Creating MyClass
__init__: Initializing MyClass
True


> **Note:** __new__ creates, __init__ initializes the class


### Enforcing Class Standards (Slide 67)


In [4]:
class RequireDocstring(type):
    """Metaclass that enforces docstrings"""
    def __new__(mcs, name, bases, attrs):
        # Check for docstring
        if '__doc__' not in attrs or not attrs['__doc__']:
            raise TypeError(f"{name} must have a docstring!")

        # Check all methods have docstrings
        for key, value in attrs.items():
            if callable(value) and not key.startswith('_'):
                if not value.__doc__:
                    raise TypeError(f"Method {key} must have docstring")

        return super().__new__(mcs, name, bases, attrs)

# This works
class GoodClass(metaclass=RequireDocstring):
    """This class has a docstring"""

    def method(self):
        """This method has a docstring"""
        pass

# This fails!
try:
    class BadClass(metaclass=RequireDocstring):
        def method(self):
            pass
except TypeError as e:
    print(e)  # BadClass must have a docstring!


BadClass must have a docstring!


> **Note:** Metaclasses can enforce coding standards


### Automatic Registration Pattern (Slide 68)


In [5]:
# Registry metaclass
class PluginRegistry(type):
    plugins = {}

    def __new__(mcs, name, bases, attrs):
        cls = super().__new__(mcs, name, bases, attrs)

        # Auto-register non-base classes
        if name != 'Plugin':
            mcs.plugins[name] = cls
            print(f"Registered plugin: {name}")

        return cls

# Base plugin class
class Plugin(metaclass=PluginRegistry):
    pass

# Auto-registered!
class ImagePlugin(Plugin):
    def process(self):
        return "Processing image"

class VideoPlugin(Plugin):
    def process(self):
        return "Processing video"

# Access registered plugins
print(PluginRegistry.plugins)
# {'ImagePlugin': <class 'ImagePlugin'>, 
#  'VideoPlugin': <class 'VideoPlugin'>}

# Use plugins dynamically
for name, plugin_class in PluginRegistry.plugins.items():
    plugin = plugin_class()
    print(f"{name}: {plugin.process()}")


Registered plugin: ImagePlugin
Registered plugin: VideoPlugin
{'ImagePlugin': <class '__main__.ImagePlugin'>, 'VideoPlugin': <class '__main__.VideoPlugin'>}
ImagePlugin: Processing image
VideoPlugin: Processing video


> **Note:** Auto-registration pattern for plugins


### Singleton with Metaclass (Slide 69)


In [6]:
class Singleton(type):
    """Metaclass that creates singleton classes"""
    _instances = {}

    def __call__(cls, *args, **kwargs):
        # __call__ is invoked when instance is created
        if cls not in cls._instances:
            # Create instance only once
            instance = super().__call__(*args, **kwargs)
            cls._instances[cls] = instance
        return cls._instances[cls]

class Database(metaclass=Singleton):
    def __init__(self):
        print("Initializing database...")
        self.connection = "DB Connection"

# First call - creates instance
db1 = Database()
# Output: Initializing database...

# Second call - returns same instance
db2 = Database()
# No output!

print(db1 is db2)  # True (same object)
print(id(db1) == id(db2))  # True

# All instances are identical
db1.connection = "Modified"
print(db2.connection)  # Modified


Initializing database...
True
True
Modified


> **Note:** Metaclass __call__ controls instance creation


### ORM-Style Metaclass (Slide 70)


In [7]:
# Simplified ORM metaclass (Django/SQLAlchemy style)
class Field:
    def __init__(self, field_type):
        self.field_type = field_type

class ModelMeta(type):
    def __new__(mcs, name, bases, attrs):
        # Collect all fields
        fields = {}
        for key, value in list(attrs.items()):
            if isinstance(value, Field):
                fields[key] = value
                attrs.pop(key)  # Remove from class attrs

        # Store fields in _fields attribute
        attrs['_fields'] = fields

        return super().__new__(mcs, name, bases, attrs)

class Model(metaclass=ModelMeta):
    pass

# Define a model
class User(Model):
    name = Field('string')
    age = Field('integer')
    email = Field('string')

# Fields are collected
print(User._fields)
# {'name': <Field>, 'age': <Field>, 'email': <Field>}

# Class is clean
print(hasattr(User, 'name'))  # False
print(hasattr(User, '_fields'))  # True


{'name': <__main__.Field object at 0x000001446F544440>, 'age': <__main__.Field object at 0x000001446F3820D0>, 'email': <__main__.Field object at 0x000001446F381E50>}
False
True


> **Note:** Metaclasses power ORMs like Django


### ABCMeta - Abstract Base Class Metaclass (Slide 71)


In [8]:
from abc import ABCMeta, abstractmethod

# ABCMeta is a metaclass!
class Shape(metaclass=ABCMeta):
    @abstractmethod
    def area(self):
        pass

# Can't instantiate
try:
    shape = Shape()
except TypeError as e:
    print(e)  # Can't instantiate abstract class

# Must implement abstract methods
class Circle(Shape):
    def __init__(self, radius):
        self.radius = radius

    def area(self):
        return 3.14 * self.radius ** 2

circle = Circle(5)
print(circle.area())  # 78.5

# Check metaclass
print(type(Shape))   # <class 'abc.ABCMeta'>
print(type(Circle))  # <class 'abc.ABCMeta'>

# ABCMeta enforces abstract method implementation


Can't instantiate abstract class Shape without an implementation for abstract method 'area'
78.5
<class 'abc.ABCMeta'>
<class 'abc.ABCMeta'>


> **Note:** ABC uses metaclass to enforce abstract methods


### Metaclass Best Practices (Slide 72)


<p><strong>When to Use Metaclasses:</strong></p>
<ul>
<li>Framework development (ORMs, APIs)</li>
<li>Automatic class registration</li>
<li>Enforcing class structure</li>
<li>Advanced class customization</li>
</ul>
<p><strong>When NOT to Use:</strong></p>
<ul>
<li>99% of the time! Metaclasses are rarely needed</li>
<li>If decorators or class inheritance work, use those instead</li>
<li>"If you wonder whether you need metaclasses, you don't" - Tim Peters</li>
</ul>
<p><strong>Alternatives:</strong></p>
<ul>
<li>Class decorators (simpler)</li>
<li><code>__init_subclass__</code> (Python 3.6+)</li>
<li>Descriptors</li>
<li>Regular inheritance</li>
</ul>
<p><strong>Remember:</strong> Metaclasses are powerful but complex. Use sparingly!</p>
